In [109]:
import pandas as pd

In [110]:
ec_list = [
    "URA_ResidentialTransaction_EC2020_20240917220317",
    "URA_ResidentialTransaction_EC2021_20240917220358",
    "URA_ResidentialTransaction_EC2022_20240917220420",
    "URA_ResidentialTransaction_EC2023_20240917220459",
    "URA_ResidentialTransaction_EC2024_20240917220523",
]

condo_list = [
    "URA_ResidentialTransaction_Conda2020_20240917220234",
    "URA_ResidentialTransaction_Conda2021_20240917220149",
    "URA_ResidentialTransaction_Conda2022_20240917220116",
    "URA_ResidentialTransaction_Conda2023_20240917215948",
    "URA_ResidentialTransaction_Condo2024_20240917215852",
]

hdb_list = [
    "ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016",
    "ResaleflatpricesbasedonregistrationdatefromJan2017onwards",
]

In [118]:
ec_df

,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Street Name,Type of Sale,Type of Area,Area (SQM),Unit Price ($ PSM),Nett Price($),Property Type,Number of Units,Tenure,Postal District,Market Segment,Floor Level
0,PARK GREEN,"870,000","1,194.8",728,Dec-20,RIVERVALE LINK,Resale,Strata,111,"7,838",-,Executive Condominium,1,99 yrs lease commencing from 2001,19,Outside Central Region,11 to 15
1,TWIN WATERFALLS,"908,000",914.94,992,Dec-20,PUNGGOL WALK,Resale,Strata,85,"10,682",-,Executive Condominium,1,99 yrs lease commencing from 2011,19,Outside Central Region,11 to 15
2,HERON BAY,"1,370,000","1,388.56",987,Dec-20,UPPER SERANGOON VIEW,Resale,Strata,129,"10,620",-,Executive Condominium,1,99 yrs lease commencing from 2012,19,Outside Central Region,11 to 15
3,ESPARINA RESIDENCES,"952,000",828.83,"1,149",Dec-20,COMPASSVALE BOW,Resale,Strata,77,"12,364",-,Executive Condominium,1,99 yrs lease commencing from 2010,19,Outside Central Region,11 to 15
4,TWIN WATERFALLS,"1,288,000","1,237.86","1,041",Dec-20,PUNGGOL WALK,Resale,Strata,115,"11,200",-,Executive Condominium,1,99 yrs lease commencing from 2011,19,Outside Central Region,11 to 15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,ECOPOLITAN,"1,390,000","1,097.93","1,266",Jan-24,PUNGGOL WALK,Resale,Strata,102,"13,627",-,Executive Condominium,1,99 yrs lease commencing from 2012,19,Outside Central Region,06 to 10
405,TREASURE CREST,"1,790,000","1,248.62","1,434",Jan-24,ANCHORVALE CRESCENT,Resale,Strata,116,"15,431",-,Executive Condominium,1,99 yrs lease commencing from 2015,19,Outside Central Region,01 to 05
406,ESPARINA RESIDENCES,"1,318,800",828.83,"1,591",Jan-24,COMPASSVALE BOW,Resale,Strata,77,"17,127",-,Executive Condominium,1,99 yrs lease commencing from 2010,19,Outside Central Region,11 to 15
407,WATERBAY,"1,598,000","1,140.98","1,401",Jan-24,EDGEFIELD PLAINS,Resale,Strata,106,"15,075",-,Executive Condominium,1,99 yrs lease commencing from 2012,19,Outside Central Region,06 to 10


In [111]:
ec_df = [pd.read_csv(f"../data/raw_data/csv/ura/{ec}.csv") for ec in ec_list]
ec_df = pd.concat(ec_df)
ec_df.to_parquet(r"../data/L1/housing_ec_transaction.parquet")

In [112]:
condo_df = [
    pd.read_csv(f"../data/raw_data/csv/ura/{condo}.csv") for condo in condo_list
]
condo_df = pd.concat(condo_df)
condo_df['Area (SQM)'] = condo_df['Area (SQM)'].str.replace(',', '').str.strip()
condo_df['Area (SQM)'] = pd.to_numeric(condo_df['Area (SQM)'], errors='coerce')
condo_df.to_parquet(r"../data/L1/housing_condo_transaction.parquet")

In [113]:
hdb_df = [pd.read_csv(f"../data/raw_data/csv/datagov/{hdb}.csv") for hdb in hdb_list]
hdb_df = pd.concat(hdb_df)

import re

def standardize_lease_duration(lease):
    if isinstance(lease, int) or lease.isdigit():
        return int(lease) * 12  # assume months
    else:
        match = re.match(r'(\d+) years?\s*', lease) #(\d+) months?
        if match:
            years = int(match.group(1))
            # months = int(match.group(2)) if match.group(2) else 0
            return years * 12 #+ months
        else:
            return None  # or raise an exception

hdb_df['remaining_lease_months'] = hdb_df['remaining_lease'].apply(standardize_lease_duration)
hdb_df.drop('remaining_lease', axis=1, inplace=True)
hdb_df.to_parquet(r"../data/L1/housing_hdb_transaction.parquet")

# Combining to idenfity all unique condo and flats

In [45]:
housing_df = pd.concat(
    [
        condo_df[["Project Name", "Street Name"]].drop_duplicates(),
        ec_df[["Project Name", "Street Name"]].drop_duplicates(),
        hdb_df[["block", "street_name"]].drop_duplicates(),
    ],
    ignore_index=True,
)

NameAddress_list = ["Project Name", "Street Name", "block", "street_name"]
for i in NameAddress_list:
    housing_df[i] = housing_df[i].fillna("")
housing_df["NameAddress"] = housing_df[NameAddress_list].agg(" ".join, axis=1)
housing_df["NameAddress"] = [i.strip() for i in housing_df["NameAddress"]]

In [95]:
for search_string in housing_df['NameAddress'][:10]:
    print(search_string)

AFFINITY AT SERANGOON SERANGOON NORTH AVENUE 1
THE FLORENCE RESIDENCES HOUGANG AVENUE 2
THE GARDEN RESIDENCES SERANGOON NORTH VIEW
THE PROMENADE@PELIKAT JALAN PELIKAT
PRIMO RESIDENCES JALAN PELIKAT
SENGKANG GRAND RESIDENCES COMPASSVALE BOW
COMPASS HEIGHTS SENGKANG SQUARE
A TREASURE TROVE PUNGGOL WALK
BOTANIQUE AT BARTLEY UPPER PAYA LEBAR ROAD
REGENTVILLE HOUGANG STREET 92


# OneMap Setup

In [79]:
import requests
import os
import json

url = "https://www.onemap.gov.sg/api/auth/post/getToken"
      
payload = {
        "email": os.environ['ONEMAP_EMAIL'],
        "password": os.environ['ONEMAP_EMAIL_PASSWORD']
      }
      
response = requests.request("POST", url, json=payload)
access_token = json.loads(response.text)['access_token']
headers = {"Authorization": f"{access_token}"}

# Search for X, Y and other data on OneMap
- this will take a while
- with exponential backoff and limit to failure

In [102]:
import requests
import json
import pandas as pd
import time
import random

df_list = []
max_retries = 3
initial_backoff = 1  # seconds
max_backoff = 32  # seconds

for search_string in housing_df['NameAddress']:
    retries = 0
    success = False
    backoff = initial_backoff

    while not success and retries < max_retries:
        try:
            url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_string}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
            response = requests.request("GET", url, headers=headers)
            response.raise_for_status()  # Raise an exception for HTTP errors

            _df = pd.DataFrame(json.loads(response.text)['results']).reset_index().rename({'index':'search_result'}, axis=1)
            _df['NameAddress'] = search_string
            df_list.append(_df)

            success = True

        except requests.RequestException as e:
            retries += 1
            backoff = min(backoff * 2, max_backoff)  # Exponential backoff
            delay = backoff + random.uniform(0, 1)  # Add some jitter to the delay
            print(f"Request failed for {search_string}. Retrying in {delay:.2f} seconds. (Retry {retries}/{max_retries})")
            time.sleep(delay)

    if not success:
        print(f"Failed to retrieve data for {search_string} after {max_retries} retries.")

In [103]:
df_housing_searched = pd.concat(df_list)
df_housing_searched.to_parquet(r"../data/L1/housing_unique_searched.parquet")

In [119]:
housing_df['NameAddress']

0       AFFINITY AT SERANGOON SERANGOON NORTH AVENUE 1
1             THE FLORENCE RESIDENCES HOUGANG AVENUE 2
2           THE GARDEN RESIDENCES SERANGOON NORTH VIEW
3                  THE PROMENADE@PELIKAT JALAN PELIKAT
4                       PRIMO RESIDENCES JALAN PELIKAT
                             ...                      
9794                             457B SENGKANG WEST RD
9795                               106B BIDADARI PK DR
9796                             458B SENGKANG WEST RD
9797                               105A BIDADARI PK DR
9798                               106A BIDADARI PK DR
Name: NameAddress, Length: 9799, dtype: object

In [117]:
df_housing_searched

,search_result,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,NameAddress
0,0,AFFINITY AT SERANGOON,26,SERANGOON NORTH AVENUE 1,AFFINITY AT SERANGOON,26 SERANGOON NORTH AVENUE 1 AFFINITY AT SERANG...,554340,32491.1666975729,38691.5738769248,1.36618702976306,103.873674807486,AFFINITY AT SERANGOON SERANGOON NORTH AVENUE 1
1,1,AFFINITY AT SERANGOON,28,SERANGOON NORTH AVENUE 1,AFFINITY AT SERANGOON,28 SERANGOON NORTH AVENUE 1 AFFINITY AT SERANG...,554341,32533.3550305134,38706.638948584,1.36632326642545,103.874053900969,AFFINITY AT SERANGOON SERANGOON NORTH AVENUE 1
2,2,AFFINITY AT SERANGOON,20,SERANGOON NORTH AVENUE 1,AFFINITY AT SERANGOON,20 SERANGOON NORTH AVENUE 1 AFFINITY AT SERANG...,554337,32487.5109304991,38622.6554335449,1.36556375666326,103.87364194752,AFFINITY AT SERANGOON SERANGOON NORTH AVENUE 1
3,3,AFFINITY AT SERANGOON,48,SERANGOON NORTH AVENUE 1,AFFINITY AT SERANGOON,48 SERANGOON NORTH AVENUE 1 AFFINITY AT SERANG...,554319,32511.4790268894,38597.4864246324,1.36533613354034,103.873857313484,AFFINITY AT SERANGOON SERANGOON NORTH AVENUE 1
4,4,AFFINITY AT SERANGOON,44,SERANGOON NORTH AVENUE 1,AFFINITY AT SERANGOON,44 SERANGOON NORTH AVENUE 1 AFFINITY AT SERANG...,554316,32522.4422722002,38629.4952411652,1.36562560811021,103.873955830609,AFFINITY AT SERANGOON SERANGOON NORTH AVENUE 1
...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,FERNVALE WOODS,457B,SENGKANG WEST ROAD,FERNVALE WOODS,457B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792457,32168.7083977796,41592.4737811014,1.39242177196708,103.870777707872,457B SENGKANG WEST RD
0,0,ALKAFF VISTA,106B,BIDADARI PARK DRIVE,ALKAFF VISTA,106B BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,342106,32211.8843447613,35092.917645948,1.33364211083176,103.871164760412,106B BIDADARI PK DR
0,0,FERNVALE WOODS,458B,SENGKANG WEST ROAD,FERNVALE WOODS,458B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792458,32104.0491439952,41649.5559899863,1.39293801208567,103.870196701638,458B SENGKANG WEST RD
0,0,ALKAFF VISTA,105A,BIDADARI PARK DRIVE,ALKAFF VISTA,105A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,341105,32130.1762556639,35133.2631429676,1.33400699237548,103.870430572231,105A BIDADARI PK DR
